In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack, clone_model

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

In [5]:
query = """
select date(time), count(*) from point
where time > '2019-11-01'
group by 1
order by 1
"""
display(pd.read_sql_query(query, engine))

,date,count
0,2019-11-01,1
1,2019-11-02,2
2,2019-11-03,1
3,2019-11-04,1
4,2019-11-05,1
5,2019-11-06,2
6,2019-11-07,1
7,2019-11-09,3
8,2019-11-10,3
9,2019-11-11,1


# Find tracks for day

In [6]:
date = "2019-11-30"

In [7]:
bracket_query = """
select track.id, min(time) as min_time, max(time) as max_time from track
join point on track_id = track.id
where track.parent_id is null
group by track.id
having abs(date(min(time)) - %(date)s) < 2  or abs(date(max(time)) - %(date)s) < 2
"""
display(pd.read_sql_query(bracket_query, engine, params={'date': date}))


,id,min_time,max_time
0,12744,2019-12-01 05:11:29+00:00,2019-12-01 05:12:36+00:00
1,1522,2019-12-01 08:54:26+00:00,2019-12-01 08:54:26+00:00
2,12692,2019-11-30 00:49:21+00:00,2019-11-30 00:54:11+00:00
3,12686,2019-11-30 00:04:24+00:00,2019-11-30 00:06:49+00:00
4,12756,2019-12-01 10:20:40+00:00,2019-12-02 00:35:14+00:00
...,...,...,...
190,2802,2019-12-01 07:37:34+00:00,2019-12-01 07:37:34+00:00
191,12675,2019-11-29 07:52:30+00:00,2019-11-29 07:59:21+00:00
192,12712,2019-11-30 06:37:32+00:00,2019-11-30 07:12:21+00:00
193,12742,2019-12-01 04:52:27+00:00,2019-12-01 04:58:22+00:00


In [8]:
bracket_tracks = [session.query(GPSTrack).get(int(t)) 
                  for t in pd.read_sql_query(bracket_query, engine, params={'date': date})[['id']].values]
len(bracket_tracks)

195

In [9]:
day_tracks = [t for t in bracket_tracks 
              if str(t.start.localtime.date()) == date or str(t.end.localtime.date()) == date]
len(day_tracks)

93

In [10]:
set([t.filename for t in day_tracks])

{'2019-11-29.json.gz',
 '2019-11-30 2.json.gz',
 '2019-12-01.json.gz',
 None,
 'inreach.gpx'}

In [11]:
set([t.name for t in day_tracks])

{'1, Nachisan',
 '156-1, Nachisan',
 '2, Tsukiji 6-Chōme',
 '396, Nachisan',
 '447, Nachisan',
 'Aneel Nazareth (9931878)',
 'Daimonzaka',
 'Hotel Kuu Kyoto (ホテル空)',
 'Kii-Katsuura Station (紀伊勝浦駅)',
 'Kumano Nachi Taisha (熊野那智大社)',
 'Kyoto Beer Lab (京都ビアラボ)',
 'Nachi Falls (那智の瀧)',
 'Nachisan',
 None,
 'RIO',
 'Seiganto-ji',
 'Shingū',
 'Shinkansen Kyoto Station (東海道新幹線 京都駅)',
 'Shinkansen Nagoya Station (東海道新幹線 名古屋駅)',
 'Three Story pagoda',
 'Unknown Place',
 'Veg Out',
 '勝浦駅 バス停',
 '南紀勝浦温泉 万清楼',
 '熊野交通 勝浦駅前出札所',
 '那智の滝前バス停'}

In [12]:
set([t.start.localtime.date() for t in day_tracks])

{datetime.date(2019, 11, 29), datetime.date(2019, 11, 30)}

# Visualization

In [13]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,
    Polyline, 
    Popup,
    GeoJSON,
    LayersControl,
    basemaps,
    CircleMarker
)
from ipywidgets import HTML

from traitlets import link, Tuple
from sidecar import Sidecar

In [14]:
def marker(t):
    marker = Marker(location=t.points[0].lat_lon)
    marker.popup = HTML(f"{t.id}: {t.name}")
    return marker

def add_track(m, t, **kwargs):
        l = t.as_polyline(
            fill=False,
            **kwargs
        )
        m.add_layer(l)
        l.popup = HTML(f"{t.id}: {t.name or t.filename}<br/>{t.points[0].time} - {t.points[-1].time}")

def bounds(lat_lons):
    lats, lons = zip(*lat_lons)
    return ((min(lats), min(lons)), (max(lats), max(lons)))
        
def center(b):
    return ((b[0][0] + b[1][0])/2, (b[0][1] + b[1][1])/2)

In [15]:
sc = Sidecar(title='Map')

m = Map(center=center(bounds([p.lat_lon for t in day_tracks for p in t.points])), 
        zoom=2, close_popup_on_click=False)
m.layer_control = LayersControl(position='topright')
m.add_control(m.layer_control)
m.target_bounds = Tuple()

markers = []
for track in day_tracks:
    if len(track.points) == 1 and track.name is not None:
        markers.append(marker(track))
    else:
        add_track(m, track)
m.add_layer(MarkerCluster(markers=markers))

with sc:
    display(m)

# Cleanup overlap

In [16]:
from intervaltree import IntervalTree, Interval
from datetime import timedelta

In [24]:
interval_tree = IntervalTree()
for t in day_tracks:
    start = t.start.time
    end = t.end.time
    if start == end:
        end += timedelta(seconds=1)
    interval_tree.addi(start, end, {t})
interval_tree.split_overlaps()
interval_tree.merge_equals(data_reducer=lambda a, b: a.union(b))

In [25]:
session.execute("delete from track where parent_id is not null")
session.commit()

In [26]:
len(interval_tree)

153

In [27]:
intervals = []
for interval in sorted(interval_tree):
    sections = [t.section(interval.begin, interval.end) for t in interval.data]
    sections = [s for s in sections if s]
    if sections:
        for s in sections:
            session.add(s)
            session.commit()
        intervals.append(Interval(interval.begin, interval.end, sections))
    else:
        print(f"empty interval {interval.begin} - {interval.end}")

empty interval 2019-11-29 23:02:14+00:00 - 2019-11-29 23:02:18+00:00
empty interval 2019-11-29 23:24:52+00:00 - 2019-11-29 23:24:55+00:00
empty interval 2019-11-29 23:57:08+00:00 - 2019-11-29 23:57:14+00:00
empty interval 2019-11-30 00:00:08+00:00 - 2019-11-30 00:00:13+00:00
empty interval 2019-11-30 00:06:49+00:00 - 2019-11-30 00:06:51+00:00
empty interval 2019-11-30 00:09:25+00:00 - 2019-11-30 00:09:29+00:00
empty interval 2019-11-30 00:49:17+00:00 - 2019-11-30 00:49:21+00:00
empty interval 2019-11-30 01:06:54+00:00 - 2019-11-30 01:06:57+00:00
empty interval 2019-11-30 01:13:05+00:00 - 2019-11-30 01:13:09+00:00
empty interval 2019-11-30 01:20:23+00:00 - 2019-11-30 01:20:26+00:00
empty interval 2019-11-30 01:27:11+00:00 - 2019-11-30 01:27:16+00:00
empty interval 2019-11-30 02:35:58+00:00 - 2019-11-30 02:37:57+00:00
empty interval 2019-11-30 03:17:52+00:00 - 2019-11-30 03:18:52+00:00
empty interval 2019-11-30 03:18:53+00:00 - 2019-11-30 03:18:56+00:00
empty interval 2019-11-30 03:23:37

In [28]:
len(intervals)

124

In [29]:
for i in sorted(intervals):
    print(f"{str(i.begin)}\t{str(i.end)}\t{i.end - i.begin}\t{' '.join(set([t.name or t.filename for t in i.data]))}")

2019-11-29 07:59:25+00:00	2019-11-29 21:21:10+00:00	13:21:45	南紀勝浦温泉 万清楼
2019-11-29 21:21:10+00:00	2019-11-29 21:21:11+00:00	0:00:01	南紀勝浦温泉 万清楼
2019-11-29 21:21:11+00:00	2019-11-29 22:54:04+00:00	1:32:53	南紀勝浦温泉 万清楼
2019-11-29 22:54:27+00:00	2019-11-29 22:55:15+00:00	0:00:48	2019-11-30 2.json.gz
2019-11-29 22:55:15+00:00	2019-11-29 23:02:14+00:00	0:06:59	2019-11-30 2.json.gz Aneel Nazareth (9931878)
2019-11-29 23:02:18+00:00	2019-11-29 23:02:19+00:00	0:00:01	Kii-Katsuura Station (紀伊勝浦駅)
2019-11-29 23:02:19+00:00	2019-11-29 23:04:46+00:00	0:02:27	Kii-Katsuura Station (紀伊勝浦駅) Aneel Nazareth (9931878)
2019-11-29 23:04:46+00:00	2019-11-29 23:04:47+00:00	0:00:01	Kii-Katsuura Station (紀伊勝浦駅) Aneel Nazareth (9931878)
2019-11-29 23:04:47+00:00	2019-11-29 23:20:59+00:00	0:16:12	Kii-Katsuura Station (紀伊勝浦駅) Aneel Nazareth (9931878)
2019-11-29 23:20:59+00:00	2019-11-29 23:21:00+00:00	0:00:01	熊野交通 勝浦駅前出札所
2019-11-29 23:21:00+00:00	2019-11-29 23:24:52+00:00	0:03:52	Kii-Katsuura Station (紀伊勝浦駅) Aneel 

# Output

In [30]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from traitlets import Tuple
from palettable.cartocolors.qualitative import Prism_10

In [31]:
# def track_points_in_interval(track, interval):
#    return [p for p in track.points if p.time >= interval.begin and p.time <= interval.end]

In [32]:
# def points_in_interval(i):
#    return (
#         session.query(GPSPoint)
#             .filter(GPSPoint.time >= i.begin)
#             .filter(GPSPoint.time <= i.end)
#             .distinct(GPSPoint.time)
#             .order_by(GPSPoint.time)
#         ).all()

In [33]:
def zoom_out_to_target_bounds(change):
    m = change.owner
    if m.zoom > 1 and m.target_bounds:
        b = m.target_bounds
        n = change.new
        if (n[0][0] < b[0][0] and n[0][1] < b[0][1] and
            n[1][0] > b[1][0] and n[1][1] > b[1][1]):
            # bounds are already large enough, so remove the target
            m.target_bounds = None
    else:
        # zoom out
        m.zoom = m.zoom - 1

In [34]:
# m = Map(center=center(bounds([p.lat_lon for p in t.points for t in day_tracks])))
m.observe(zoom_out_to_target_bounds, 'bounds')

default_bounds = bounds([p.lat_lon for p in t.points for t in day_tracks])

#m
#with sc:
#    display(m)

In [35]:
def show_tracks_in_interval(m, i):
    print(f"{str(i.begin)}\t{str(i.end)}\t{i.end - i.begin}")
    
    b = []
    markers = []
    lines = []
    n = 0
    for t in i.data:
        points = t.points # track_points_in_interval(t, i)
        if not points:
            continue
        print(f"{t.id}\t{points[0].localtime}\t{points[-1].localtime}\t{points[-1].time - points[0].time}\t{t.name}\t{t.filename}")
        color = Prism_10.hex_colors[n % len(Prism_10.hex_colors)]
        lat_lons = [p.lat_lon for p in points]
        b = bounds(lat_lons + [p for p in b])
        if len(points) == 1:
            marker = CircleMarker(location=points[0].lat_lon, color=color)
            marker.name = f"{t.id}: {t.name or t.filename or 'Unnamed'}"
            marker.popup = HTML(f"{t.id}: {t.name or t.filename}<br/>{points[0].time}")
            markers.append(marker)
        else:
            line = Polyline(locations=lat_lons, fill=False, color=color)
            line.name = f"{t.id}: {t.name or t.filename or 'Unnamed'}"
            line.popup = HTML(f"{t.id}: {t.name or t.filename}<br/>{t.points[0].time} - {t.points[-1].time}")
            lines.append(line)
        n += 1
    
    m.target_bounds = b or default_bounds
    m.center = center(b or default_bounds)
    m.zoom = 18
     
    m.clear_layers()
    m.add_layer(basemaps.OpenStreetMap.Mapnik)
    if markers:
        m.add_layer(MarkerCluster(markers=markers))
    for line in lines:
        m.add_layer(line)
    if i.data:
        query = """
        select id, name, description, source, type, filename from track where id in %(ids)s
        """
        display(pd.read_sql_query(query, engine, params={'ids': tuple([t.id for t in i.data])}))
    return i
    

In [36]:
w = interactive(lambda i: show_tracks_in_interval(m, i), 
                i=[(str(i.begin.time()), i) for i in intervals])
w

interactive(children=(Dropdown(description='i', options=(('07:59:25', Interval(datetime.datetime(2019, 11, 29,…

In [ ]:
i = w.result

In [ ]:
i

In [ ]:
[t.id for t in i.data]

In [ ]:
[len(t.points) for t in i.data]

In [ ]:
removed = i.data.pop(-1)

In [ ]:
if not i.data:
    intervals.remove(i)
len(intervals)

In [ ]:
[len(i.data) for i in sorted(intervals)]

In [ ]:
[str(t) for t in sorted(intervals)[0].data]

In [ ]:
str(sorted(intervals)[0].begin)

In [ ]:
str(sorted(intervals)[0].end)

In [ ]:
m, b = w.result

In [ ]:
sc = Sidecar(title='Map')
with sc:
    display(m)

In [ ]:
[str(p.track) for p in (session.query(GPSPoint)
            .filter(GPSPoint.time >= "2019-11-29 23:02:14+00:00")
            .filter(GPSPoint.time <= "2019-11-29 23:02:18+00:00")
            .distinct(GPSPoint.time)
            .order_by(GPSPoint.time)
        ).all()]

In [ ]:
m.target_bounds

In [ ]:
m, b = w.result
for z in range(18, 1, -1):
    m.zoom = z
    print(f"{z}\t{m.bounds}")
    if (m.bounds[0][1] < b[0] and m.bounds[0][0] < b[1] and
        m.bounds[1][1] > b[2] and m.bounds[1][0] > b[3]):
        break

In [ ]:
m.zoom = 17
m.bounds

In [ ]:
import gpxpy
from geopy.distance import distance

In [ ]:
def points_to_gpx_track(points, tracks):
    t = gpxpy.gpx.GPXTrack(
            name=f"{points[0].localtime.time()} {' '.join({t.type or '' for t in tracks})}",
            description="\t".join({t.description or "" for t in tracks}),
        )
    t.comment="\t".join({t.name or t.filename or "" for t in tracks})
    t.source="\t".join({t.source or "" for t in tracks})
    t.type="\t".join({t.type or "" for t in tracks})
    s = gpxpy.gpx.GPXTrackSegment()
    t.segments.append(s)
    for p in points:
        s.points.append(gpxpy.gpx.GPXTrackPoint(
            latitude=p.latitude, longitude=p.longitude, elevation=p.elevation, time=p.time))
    return t

In [ ]:
HOUR = timedelta(hours=1)

def max_speed(points):
    max_s = 0
    for j in range(1, len(points)):
        d = distance((points[j-1].latitude, points[j-1].longitude),
                     (points[j].latitude, points[j].longitude)).miles
        t = (points[j].time - points[j-1].time) / HOUR
        s = d/(t + 1/3600)
        max_s = max(max_s, s)
    return max_s


In [ ]:
gpx = gpxpy.gpx.GPX()
for i in sorted(intervals):
    track_ids = [t.id for t in i.data]
    track_points = dict()
    
    for track_id in track_ids:
        points = (
            session.query(GPSPoint)
            .filter(GPSPoint.track_id.in_(track_ids))
            .filter(GPSPoint.time.between(i.begin, i.end))
            .distinct(GPSPoint.time)
            .order_by(GPSPoint.time)
        ).all()
        if points:
            track_points[track_id] = points
            
    track_ids = sorted(track_points.keys())
    if not track_ids:
        continue
    merged = track_points[track_ids.pop(0)]
    merged_speed = max_speed(merged)
    unmerged = []
    while track_ids:
        next_id = track_ids.pop(0)
        potential_merge = sorted(merged + track_points[next_id], key=lambda p: p.time)
        potential_speed = max_speed(potential_merge)
        print(f"previous speed {merged_speed}\tnew speed {potential_speed}\tratio {potential_speed/(merged_speed + 0.0001)}")
        if potential_speed/(merged_speed + 0.0001) < 1.1:
            merged = potential_merge
            merged_speed = potential_speed
        else:
            print("### NOT MERGING")
            unmerged.append(track_points[next_id])
    
    for points in [merged] + unmerged:
        if len(points) == 1:
            p = points[0]
            t = p.track
            w = gpxpy.gpx.GPXWaypoint(
                latitude=p.latitude, longitude=p.longitude, elevation=p.elevation,
                time=p.time, name=t.name, description=t.description, type=t.type,
                comment=t.source or t.filename
            )
            gpx.waypoints.append(w)
        else:
            tracks = {p.track for p in points}
            #if len(tracks) > 1:
            #    total_d = 0
            #    max_s = 0
            #    one_hour = timedelta(hours=1)
            #    for j in range(1, len(points)):
            #        d = distance((points[j-1].latitude, points[j-1].longitude),
            #                     (points[j].latitude, points[j].longitude)).miles
            #        total_d += d
            #        t = (points[j].time - points[j-1].time) / one_hour
            #        s = d/t
            #        # print(d, t, s)
            #        max_s = max(max_s, s)
            #    total_t = (points[-1].time - points[0].time) / one_hour
            #    avg_s = total_d / total_t
            #    print(f"max speed: {max_s}  avg_speed: {avg_s}")
            #    # speed is too fast, might be pingponging between two tracks, include both
            #    if max_s > 100:
            #        for track in tracks:
            #            track_id = track.id
            #            track_points = [p for p in points if p.track_id == track_id]
            #            gpx.tracks.append(points_to_gpx_track(track_points, [track]))
            #            print("filtered track")
            #            gpx.tracks[-1].number = len(gpx.tracks)
            gpx.tracks.append(points_to_gpx_track(points, tracks))
            gpx.tracks[-1].number = len(gpx.tracks)
    print(f"{str(points[0].localtime)}\t{str(points[-1].localtime.time())}\t{i.end - i.begin}\t{len(points)}\t{set(t.name or t.filename for t in i.data)}")

    

In [ ]:
with open(f"{date}.gpx", "w") as fh:
    print(gpx.to_xml(version="1.1"), file=fh)

In [ ]:
for t in session.query(GPSTrack):
    if len(t.points) > 1:
        print(f"{t.id}\t{max_speed(t.points)}\t{t.name}")

In [ ]:
speed_query = """
select time as t1, lag(time) over (order by time) as t2,
    latitude as lat1, lag(latitude) over (order by time) as lat2,
    longitude as lon1, lag(longitude) over (order by time) as lon2
from point
where track_id = %(track_id)s
order by time
"""
speed_df = pd.read_sql_query(speed_query, engine, params={'track_id': 12566})
display(speed_df)

In [ ]:
speed_df["distance"] = speed_df.dropna().apply(lambda row: distance((row['lat1'], row['lon1']), (row['lat2'], row['lon2'])).miles , axis=1)

In [ ]:
speed_df

In [ ]:
speed_df["hours"] = (speed_df["t1"] - speed_df["t2"]) / HOUR

In [ ]:
speed_df = speed_df[speed_df.hours > 0].copy()

In [ ]:
speed_df["speed"] = speed_df["distance"] / speed_df["hours"]

In [ ]:
speed_df.plot.line(x="t1", y="speed")

In [ ]:
speed_df.sort_values(by="speed")

In [ ]:
speed_df.speed.plot.hist()

In [ ]:
from palettable.cartocolors.qualitative import Prism_10

In [ ]:
import palettable

In [ ]:
Prism_10.hex_colors

# Cleanup identical tracks

In [ ]:
from collections import defaultdict
length_query = """
select length, array_agg(track_id) as track_ids,
array_agg(start) as starts,
array_agg(stop) as stops
from (
select track_id, count(*) as length, min(time) as start, max(time) as stop 
from point
group by track_id) x
group by length
order by length
"""
length_df = pd.read_sql_query(length_query, engine)
length_df

In [ ]:
def points_are_same(p1, p2):
    return (p1.time == p2.time and
            p1.latitude == p2.latitude and
            p1.longitude == p2.longitude and
            p1.elevation == p2.elevation)

In [ ]:
# why do we have multiple points in the same second?
def ultrasort(points):
    return sorted(points, key=lambda p: (p.time, p.latitude, p.longitude, p.elevation))

In [ ]:
dupes = []
for row in length_df.itertuples():
    track_ids = row.track_ids
    starts = row.starts
    stops = row.stops
    by_times = defaultdict(list)
    for (start, stop, track_id) in zip(starts, stops, track_ids):
        by_times[(start, stop)].append(track_id)
    # print(f"{len(track_ids)} tracks with {len(by_times)} unique time bounds")
    for bounds, tracks_ids in by_times.items():
        if len(tracks_ids) > 1:
            tracks_ids.sort()
            uniques = [session.query(GPSTrack).get(tracks_ids[0])]
            for track_id in tracks_ids[1:]:
                t = session.query(GPSTrack).get(track_id)
                for u in uniques:
                    
                    for p1, p2 in zip(ultrasort(u.points), ultrasort(t.points)):
                        if not points_are_same(p1, p2):
                            break # not the same as this unique
                    else:
                        # all points matched, so this is a duplicate
                        print(f"{row.length}: {track_id} is a duplicate of {u.id}")
                        print(f"{u.id}\t{u.name}\t{u.filename}\t{u.source}\t{u.parent}")
                        print(f"{t.id}\t{t.name}\t{t.filename}\t{t.source}\t{t.parent}")
                        print()
                        dupes.append(t)
                        break
                else:
                    # no u matched
                    uniques.append(t)


In [ ]:
[t.id for t in dupes]

In [ ]:
for t in dupes:
    session.delete(t)

In [ ]:
session.execute("delete from track where id = 1417")

In [ ]:
session.commit()

In [ ]:
query = """
select * from track where id in (15100, 15099)
"""
pd.read_sql_query(query, engine)

In [ ]:
query = """
select * from track where id in (2492, 12385)
"""
pd.read_sql_query(query, engine)

In [ ]:
intvs = [i for i in intervals if 15099 in [t.id for t in i.data] or 15100 in [t.id for t in i.data]]

In [ ]:
intvs

In [ ]:
a = session.query(GPSTrack).get(2492)
b = session.query(GPSTrack).get(12385)

In [ ]:
len(a.points), str(a.start.time), str(a.end.time)

In [ ]:
len(b.points), str(b.start.time), str(b.end.time)

In [ ]:
for p1, p2 in zip(a.points, b.points):
    if not points_are_same(p1, p2):
        print(p1.time, p2.time,
            p1.latitude, p2.latitude,
            p1.longitude, p2.longitude,
            p1.elevation, p2.elevation)